In [4]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [5]:
#Conexao a db
database = 'database.sqlite'
con = sqlite3.connect(database)

In [24]:
#Get main tables from db
countries = pd.read_sql_query("SELECT * from Country", con)
matches = pd.read_sql_query("SELECT * from Match", con)
leagues = pd.read_sql_query("SELECT * from League", con)
teams = pd.read_sql_query("SELECT * from Team", con)
teams_att = pd.read_sql_query("SELECT * FROM Team_Attributes", con)

In [23]:
#Replace league_id for its name to make it easier to analyse the data.
league_id_map = leagues.set_index('id').to_dict()['name']
matches['league_id'] = matches['league_id'].map(league_id_map)

#Replace country_id for its name.
country_id_map = countries.set_index('id').to_dict()['name']
matches['country_id'] = matches['country_id'].map(country_id_map)

#Replace team_id for its name.
team_id_map = teams.set_index('team_api_id').to_dict()['team_long_name']
matches['home_team_api_id'] = matches['home_team_api_id'].map(team_id_map)
matches['away_team_api_id'] = matches['away_team_api_id'].map(team_id_map)

In [ ]:
# Goals avg per league
avg_goals_league = matches[['league_id','home_team_goal','away_team_goal']].groupby('league_id').mean()


total_goals = avg_goals_league.home_team_goal + avg_goals_league.away_team_goal

avg_goals_league.home_team_goal = avg_goals_league.home_team_goal / total_goals * 100
avg_goals_league.away_team_goal = avg_goals_league.away_team_goal / total_goals * 100

avg_goals_league

# Conclusion: home teams have advantage

In [15]:
# Home win 

goals_per_match = matches[['home_team_goal','away_team_goal']]
wins = 0
draws = 0

for index, row in goals_per_match.iterrows():
    if row['home_team_goal'] > row['away_team_goal']:
        wins += 1
    if row['home_team_goal'] == row['away_team_goal']:
        draws += 1
        

total_matches = len(goals_per_match)
loss = total_matches - wins - draws

percs = [wins/total_matches*100,draws/total_matches*100, loss/total_matches*100]
percs

#Conclusion : this confirms that "home team wins about 46% of the time"

[45.87166557604219, 25.389737865198814, 28.738596558759]

In [30]:
games = pd.read_sql_query("SELECT home_team_api_id, away_team_api_id, home_team_goal, away_team_goal from Match", con)
team_id_map = teams.set_index('id').to_dict()['team_long_name']
matches['home_team'] = matches['home_team_api_id'].map(team_id_map)
matches

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,home_team
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,NaN,9993,1,...,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20,NaN
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,NaN,9994,0,...,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60,NaN
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,NaN,8635,0,...,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75,NaN
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,NaN,9998,5,...,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50,NaN
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,NaN,9985,1,...,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,NaN,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,NaN,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,NaN,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,NaN,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
